In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# LLM이 실수를 일으키는 prompt
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

AIMessage(content='= $4923.38')

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agnet = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, tools=[
        Tool.from_function(
            func=plus,
            name="sum caculator",
            description="두개의 숫자를 더하는 toll 입니다. 이 tool은 콤마(,)로 구분되어 있는 숫자를 쌍으로 보낼 때 이 툴을 사용하세요. \n예시: 1,2"
        )
    ])

# LLM이 실수를 일으키는 prompt
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agnet.invoke(prompt)



> Entering new AgentExecutor chain...
I need to add up all the given numbers to find the total cost.
Action: sum calculator
Action Input: 355.39,924.87
Observation: sum calculator is not a valid tool, try one of [sum caculator].
Thought:I need to use the sum caculator tool to add up the numbers.
Action: sum caculator
Action Input: 355.39,924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total.
Action: sum caculator
Action Input: 1280.26,721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total.
Action: sum caculator
Action Input: 2001.46,1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total.
Action: sum caculator
Action Input: 3941.75,573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the total.
Action: sum caculator
Action Input: 4515.38,65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the t

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool, BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from typing import Type

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="First number")
    b: float = Field(description="Second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    두가지 숫자들의 총합을 구할때 이 도구를 사용하세요.
    첫번째, 두번째 args들은 숫자 형태여야 합니다.
    오직 두가지의 args만 받습니다.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agnet = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        CalculatorTool(),
    ],
)

# LLM이 실수를 일으키는 prompt
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agnet.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}